_Autor:_    __Jesús Casado__ <br> _Revisión:_ __26/03/2020__ <br>

__Introducción__<br>
Funciones para extraer datos de la base de datos [GCP_PPP-2005](http://doi.org/10.5880/pik.2017.007), que contiene el producto interior bruto a escala global para cada década desde 1850 a 2100.

* Sistema de coordenadas: GC WGS84
* Resolución: 0.083333º

__Cosas que arreglar__ <br>

***

__Índice__ <br>

In [1]:
import numpy as np
import pandas as pd
import datetime
import os
from netCDF4 import Dataset
from pyproj import Proj, transform
os.environ['PROJ_LIB'] = r'C:\Anaconda3\pkgs\proj4-4.9.3-vc14_5\Library\share'

In [2]:
def extract_GCP(path, lonlim=None, latlim=None, coordsIn='epsg:4326', datelim=None, dateformat='%Y-%m-%d',
                coordsOut=None, plot=False):
    """Extrae los datos de la base de datos global de PID GCP_PPP-2005 para las coordenadas y fechas indicadas.
    
    Entradas:
    ---------
    path:        string. Ruta donde se encuentran el archivo 'GCP_PPP-2005_1850-2100.nc'de GCP_PPP-2005
    lonlim:      list(2,1). Límites de la longitud (º) del rectángulo de búsqueda. Si es 'None' se extraen todas las columnas disponibles
    latlim:      list(2,1). Límites de la latitud (º) del rectángulo de búsqueda. Si es 'None' se extraen todas las filas disponibles
    coordsIn:    string. Código EPSG del sistema de coords de 'lonlim' y 'latlim'. Por defecto ED50-30N, el sistema de coordenadas de SPREAD
    datelim:     list of strings (2,1). Fechas límites de la serie a extraer. Si es 'None' se extraen todas las fechas disponibles
    dateformat:  string. Formato de las fechas en 'datelim'
    coordsOut:   string. Código EPSG del sistema de coords de los datos de salida. Por defecto las mismas que 'coordsIn'
    plot:        boolena. Si se quiere plotear la serie
    
    Salidas:
    --------
    Como métodos de la función:
    data:        array 2D [lat, lon] o 3D [dates, lat, lon]. Datos extraídos
    dates:       array 1D. Fechas en 'data'
    lat:         array 1D. Latitudes en 'data'
    lon:         array 1D. Longitudes en 'data'
    """
    
    # cargar NetCDF
    nc = Dataset(path + 'GCP_PPP-2005_1850-2100.nc', 'r', format='HDF5')

    # datos de 'GDP'
    data = nc['gdp'][::].copy()

    #time
    time = nc.variables['time']
    # convertir en fechas; la variable 'time' son los días desde el 1-ene-1661
    start = datetime.date(1661, 1, 1)
    dates = np.array([datetime.timedelta(t) + start for t in time[:]])

    # extraer fechas del periodo de interés
    if datelim != None:
        datelim = [datetime.datetime.strptime(lim, dateformat).date() for lim in list(datelim)]
        if len(datelim) == 1:
            maskdate = dates == dates[np.argmin(abs(dates - datelim))]
        elif len(datelim) == 2:
            maskdate = (dates >= datelim[0]) & (dates <= datelim[-1])
        elif len(datelim) > 2:
            print('ERROR. Longitud errónea de "datelim".')
            return
        data = data[maskdate, :, :]
        extract_GCP.dates = dates[maskdate]
    else:
        extract_GCP.dates = dates

    # comprobaciones previas: 
    if (lonlim != None) & (latlim != None):
        if len(lonlim) != 2 or len(latlim) != 2:
            print('ERROR. Longitud errónea de "latlim" o "lonlim".')
            return
        if coordsIn != 'epsg:4326':
            GC_WGS84 = Proj(init='epsg:4326')
            SistRefIn = Proj(init=coordsIn)
            lonlim, latlim = transform(SistRefIn, GC_WGS84, lonlim, latlim)

    # longitud
    lon = nc['lon'][::].data #lon = nc.variables['lon']
    if lonlim != None:
        masklon = (lon >= lonlim[0]) & (lon <= lonlim[-1])
        data = data[:, :, masklon]
        lon = lon[masklon]

    # latitud
    lat = nc['lat'][::].data #lat = gcp_ppp.variables['lat']
    if latlim != None:
        masklat = (lat >= latlim[0]) & (lat <= latlim[-1])
        data = data[:, masklat, :]
        lat = lat[masklat]

    # enmascarar PIB nulo
    data = np.ma.masked_where(data == 0, data)
    extract_GCP.data = data

    # guardar latitud y longitud en el sistema 'coords'
    if coordsOut == None:
        coordsOut = coordsIn
    if coordsOut != 'epsg:4326':
        # matrices de longitud y latitud de cada una de las celdas
        lonAux, latAux = np.meshgrid(lon, lat)
        # transformar coordendas y reformar en matrices del mismo tamaño que el mapa diario
        SistRefOut = Proj(init=coordsOut)
        lonTrans, latTrans = transform(GC_WGS84, SistRefOut, lonAux.flatten(), latAux.flatten())
        extract_GCP.lon = lonTrans.reshape(lonAux.shape)
        extract_GCP.lat = latTrans.reshape(latAux.shape)
    else:
        extract_GCP.lon = lon
        extract_GCP.lat = lat
    
    if plot == True:
        plt.imshow(np.nanmean(data, axis=0)[::-1], cmap='viridis')
        plt.axis('off');
        plt.axis('equal');